In [1]:
from hexathello import aiPlayers, autoPlayer, engine, history, jable, printing

import numpy as np

from os import path

# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
# We want to get some baseline data for learning, using a Greendom Agent, which is smarter
#   than a simple random agent
# We store this at `data/histories/greendom_size-5_players-2.json`
histories_dir: str = path.join(
    'data',
    'history',
    'examples'
)
assert path.isdir(
    histories_dir
)

baseline_data_path: str = path.join(
    histories_dir,
    'greendom_size-{}_players-{}.json'.format(
        game_size, player_count
    )
)
baseline_data: jable.JyFrame

max_data: int = 20000
existing_data: int = 0
if path.isfile( baseline_data_path ):
    print("# Reading data from {}".format( baseline_data_path ))
    baseline_data: jable.JyFrame = jable.read_file(
        baseline_data_path
    )
    existing_data = len( baseline_data )
#
else:
    print("# Starting new data at {}".format( baseline_data_path ))
    baseline_data: jable.JyFrame = history.new_literalHistory(
        player_count = player_count,
        size = game_size
    )
    baseline_data = history.history_asInt( baseline_data )
    
    # We need to change scores and winner to be shift
    baseline_data.makeColumn_shift( "winner" )
    baseline_data.makeColumn_shift( "scores" )
#/if path.isfile( baseline_data_path )/else

ai_0: aiPlayers.GreedyHexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 0,
    p_random = 0.4
)

ai_1: aiPlayers.GreedyHexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 1,
    p_random = 0.4
)
    
play_index: int = 0
if existing_data >= max_data:
    print("# Already have existing_data={}".format( existing_data) )
#

while existing_data < max_data:
    print("# Starting play_index={}".format(play_index))
    literalHistory: jable.JyFrame = autoPlayer.runHexathello_withAgents(
        agents = [ai_0, ai_1],
        size = game_size,
        logging_level = 0
    )
    
    baseline_data.extend(
        history.history_asInt(
            literalHistory
        )
    )
    
    baseline_data.write_file(
        fp = baseline_data_path
    )
    
    existing_data = len( baseline_data )
    play_index += 1
    
    # Safety Valve
    if play_index > max_data:
        raise Exception("Too large play_index={}".format(play_index))
    #
#/while existing_data < max_data

printing.prettyprint( baseline_data[:50] )

# Reading data from data/history/examples/greendom_size-5_players-2.json
# Already have existing_data=20023
player_count size history_type          ai_id action_tags turn_index current_player                          board_state     action_choices      player_action winner   scores
------------ ---- ------------ -------------- ----------- ---------- -------------- ------------------------------------ ------------------ ------------------ ------ --------
           2    5      literal GreedyHexAgent  ['random']          0              1             453356693984287226068992     71605704212480              16384      1 [20, 22]
           2    5      literal GreedyHexAgent  ['random']          1              0             453356693979889447993344     71468256854144     70368744177664      1 [20, 22]
           2    5      literal GreedyHexAgent  ['random']          2              1         9904275902431925745934663680   9008298766368768      1099511627776      1 [20, 22]
           2    5

In [3]:
# In `quickstart_training` we'll use data we created to train a smarter AI